In [1]:
import pandas as pd

# ============================
# 1. Load data
# ============================

# Actual local path to the Excel file
file_path = "Raw Data Set - final.xlsx"

# Read the main sheet
df = pd.read_excel(file_path, sheet_name="Dataset")

# Quick structure check: shape, column info, and sample rows
print("Shape (rows, cols):", df.shape)
print("\nColumn info:")
print(df.info())
print("\nSample rows:")
print(df.head())

# ============================
# 2. Fix data types
# ============================

# Convert date columns to proper datetime format
date_cols = [
    "Application Creation Date",
    "App Completion Date",
    "Action Date",
]

for col in date_cols:
    # errors="coerce" converts invalid dates to NaT instead of raising errors
    df[col] = pd.to_datetime(df[col], errors="coerce")

# Ensure funnel flag columns are numeric (0/1)
funnel_flag_cols = ["Applied", "Completed", "Admitted", "Matric", "Enrolled"]

for col in funnel_flag_cols:
    if col in df.columns:
        # Convert to numeric; non-numeric -> NaN, then fill NaN with 0 and cast to int
        df[col] = pd.to_numeric(df[col], errors="coerce").fillna(0).astype(int)

# ============================
# 3. Basic data quality checks
# ============================

# 3a. Check for duplicate Application Numbers
if "Application Number" in df.columns:
    dup_counts = df["Application Number"].duplicated().sum()
    print(f"\nNumber of duplicate Application Numbers: {dup_counts}")

# 3b. Missing values per column
print("\nMissing values per column:")
print(df.isna().sum().sort_values(ascending=False))

# 3c. Simple date sanity check:
# count rows where completion date is earlier than creation date
if set(["Application Creation Date", "App Completion Date"]).issubset(df.columns):
    mask_completion_before_creation = (
        df["App Completion Date"].notna()
        & df["Application Creation Date"].notna()
        & (df["App Completion Date"] < df["Application Creation Date"])
    )
    print(
        "\nRows where completion date is before creation date:",
        mask_completion_before_creation.sum()
    )

# ============================
# 4. Feature engineering for funnel
# ============================

# 4a. Days from application creation to completion
if set(["Application Creation Date", "App Completion Date"]).issubset(df.columns):
    df["days_to_complete"] = (
        df["App Completion Date"] - df["Application Creation Date"]
    ).dt.days

# 4b. Days from application creation to action (decision/review/etc.)
if set(["Application Creation Date", "Action Date"]).issubset(df.columns):
    df["days_to_action"] = (
        df["Action Date"] - df["Application Creation Date"]
    ).dt.days

# 4c. Application month (useful for grouping/time patterns)
if "Application Creation Date" in df.columns:
    df["app_month"] = df["Application Creation Date"].dt.to_period("M").astype(str)

print("\nFeature columns created: days_to_complete, days_to_action, app_month")

# ============================
# 5. Quick funnel-level EDA (optional early peek)
# ============================

# Overall counts at each funnel stage (you can later extend this by College/Major)
funnel_summary = {
    "Applied": df["Applied"].sum(),
    "Completed": df["Completed"].sum(),
    "Admitted": df["Admitted"].sum(),
    "Matric": df["Matric"].sum(),
    "Enrolled": df["Enrolled"].sum(),
}
print("\nFunnel stage counts (overall):")
for stage, count in funnel_summary.items():
    print(f"  {stage}: {count}")

# ============================
# 6. Save cleaned data for later analysis/visualization
# ============================

# 1) Parquet (efficient, for Python work)
output_parquet = "cleaned_recruitment_data.parquet"
df.to_parquet(output_parquet, index=False)
print(f"Parquet saved to: {output_parquet}")

# 2) CSV (easy to inspect anywhere)
output_csv = "cleaned_recruitment_data.csv"
df.to_csv(output_csv, index=False)
print(f"CSV saved to: {output_csv}")

# 3) Excel (for manual review / sharing)
output_excel = "cleaned_recruitment_data.xlsx"
df.to_excel(output_excel, index=False)
print(f"Excel saved to: {output_excel}")



Shape (rows, cols): (4566, 19)

Column info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4566 entries, 0 to 4565
Data columns (total 19 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Application Number         4566 non-null   int64         
 1   Application Creation Date  4566 non-null   datetime64[ns]
 2   App Completion Date        3279 non-null   datetime64[ns]
 3   Action Date                4566 non-null   datetime64[ns]
 4   Admit Term                 4566 non-null   int64         
 5   Session                    4566 non-null   int64         
 6   Admit Type                 4566 non-null   object        
 7   Prog Status                4566 non-null   object        
 8   Prog Actn                  4566 non-null   object        
 9   Action Rsn                 1490 non-null   object        
 10  Applied                    4566 non-null   int64         
 11  Completed               

In [7]:
# ============================
# 7. Funnel conversion by College
# ============================

# Keep only rows where Applied == 1 (core funnel)
df_applied = df[df["Applied"] == 1].copy()

# Group by College and aggregate funnel counts
college_funnel = (
    df_applied
    .groupby("College")[["Applied", "Completed", "Admitted", "Matric", "Enrolled"]]
    .sum()
)

# Compute conversion rates at each step (relative to Applied)
college_funnel["completed_rate"] = college_funnel["Completed"] / college_funnel["Applied"]
college_funnel["admitted_rate"]  = college_funnel["Admitted"] / college_funnel["Applied"]
college_funnel["matric_rate"]    = college_funnel["Matric"]   / college_funnel["Applied"]
college_funnel["enrolled_rate"]  = college_funnel["Enrolled"] / college_funnel["Applied"]

print("\nFunnel by College (counts + rates):")
print(college_funnel.sort_values("enrolled_rate", ascending=False).head(10))


# ============================
# 8. Funnel conversion by Admit Type
# ============================

admit_funnel = (
    df_applied
    .groupby("Admit Type")[["Applied", "Completed", "Admitted", "Matric", "Enrolled"]]
    .sum()
)

admit_funnel["completed_rate"] = admit_funnel["Completed"] / admit_funnel["Applied"]
admit_funnel["admitted_rate"]  = admit_funnel["Admitted"] / admit_funnel["Applied"]
admit_funnel["matric_rate"]    = admit_funnel["Matric"]   / admit_funnel["Applied"]
admit_funnel["enrolled_rate"]  = admit_funnel["Enrolled"] / admit_funnel["Applied"]

print("\nFunnel by Admit Type (counts + rates):")
print(admit_funnel.sort_values("enrolled_rate", ascending=False))





# ============================
# 9. Overall funnel summary for Tableau
# ============================

import pandas as pd

summary = pd.DataFrame({
    "Funnel Stage": [
        "01 Applied",
        "02 Completed",
        "03 Admitted",
        "04 Matric",
        "05 Enrolled",
    ],
    "Count": [
        df["Applied"].sum(),
        df["Completed"].sum(),
        df["Admitted"].sum(),
        df["Matric"].sum(),
        df["Enrolled"].sum(),
    ],
})

# Export your college_funnel, admit_funnel and overall funnel tables directly from Python
college_funnel.to_excel("college_funnel_summary.xlsx")
admit_funnel.to_excel("admit_funnel_summary.xlsx")
summary.to_excel("overall_funnel_summary.xlsx", index=False)
print(summary)






Funnel by College (counts + rates):
                               Applied  Completed  Admitted  Matric  Enrolled  \
College                                                                         
College of Education                34         24        22      20        14   
College of Applied Sci & Tech      851        569       529     467       342   
Undergrad Non-Degree Seeking       324        259       252     186       127   
College of Medicine                 24         15        15      13         9   
Eller College of Management        830        510       489     400       250   
Ag, Life & Envi Sci., Col of       274        175       171     145        82   
College of Science                 791        429       416     347       218   
College of Fine Arts                95         49        49      40        26   
College of Arch, Plan & Lands       66         43        41      38        18   
College of Soc & Behav Sci         659        327       317     254     